## copyreg를 사용해 pickle을 더 신뢰성있게 만들어라
- pickle 내장 모듈을 사용하면 파이썬 객체를 바이트 스트림으로 직렬화 하거나 바이트 스트림을 파이썬 객체로 역직렬화 할 수 있다.
- 피클된 파이트 스트림은 서로 신뢰할 수 없는 당사자 사이의 통신에 사용하면 안된다.
- 피클의 목적은 프로그램들 사이 이진 채널을 통해 서로 파이썬 객체를 넘기는데 있다.

In [2]:
import pickle

class GameState:
    def __init__(self):
        self.level = 0
        self.lives = 4

character1 = GameState()
character1.level +=5
character1.lives =1

state_path = 'game_stat.bin'
with open(state_path, 'wb') as f:
    pickle.dump(character1, f)

with open(state_path, 'rb') as f:
    state_after = pickle.load(f)

print(state_after.__dict__)

{'level': 5, 'lives': 1}


In [8]:
class GameState:
    def __init__(self):
        self.level = 0
        self.lives = 4
        self.points = 0

with open(state_path, 'rb') as f:
    state_after = pickle.load(f)

print(state_after.__dict__)

assert isinstance(state_after, GameState)
# state_after에는 이전 객체 정보만 담겨 있지만 클래스 자체는 동일하다고 판단한다.


{'level': 5, 'lives': 1}


#### 디폴트 애트리뷰터 값
- 생성자의 디폴트 값을 지정한다.
- unpickle하는 도우미 함수를 정의해서 역질려화한 객체를 인자로 하는 새로운 객체를 생성한다.
- 이떄 디폴트 값이 있기 떄문에 애트리뷰트가 사라지지 않는다.

In [13]:
class GameState:
    def __init__(self, level=0, lives=4, points=0):
        self.level = 0
        self.lives = 4
        self.points = 0

def unpickle_game_state(kwargs):
    return GameState(**kwargs)

with open(state_path, 'rb') as f:
    state_after = pickle.load(f)
    kwargs = state_after.__dict__
    state_after = unpickle_game_state(kwargs)

print(state_after.__dict__)
assert isinstance(state_after, GameState)


{'level': 0, 'lives': 4, 'points': 0}


#### 클래스 버전 지정
- 객체의 필드가 삭제 되는 경우에는 디폴트 인자를 사용하는 방법으로 접근할 수 없다.
- 역직렬화시 버전을 확인후 버전에 맞는 방식으로 객체를 생성한다.

In [22]:
class GameState:
    def __init__(self, level=0):
        self.level = 0

def unpickle_game_state(kwargs):
    version = kwargs.pop('version', 1)
    if version == 0:
        del kwargs['lives']
    return GameState(**kwargs)

with open(state_path, 'rb') as f:
    state_after = pickle.load(f)
    kwargs = state_after.__dict__
    kwargs['version'] = 0
    state_after = unpickle_game_state(kwargs)

print(state_after.__dict__)
assert isinstance(state_after, GameState)


{'level': 0}


#### 안정적인 임포트 경로
- 클래스 이름이 바뀌어 코드가 깨지는 경우를 들 수 있다.
- 클래스명이 변경 되거나 클래스가 다른 모듈로 옮겨지는 경우 가 있다.
- 이름이 변경 되는 경우 역직렬화시 기존 클래스가 없기 때문에 에러를 발생한다.
- 피클된 데이터 안에 직렬화한 클래스의 임포트 경로가 들어 있기 때문이다.
- copyreg를 쓰는 것이 해결 방법이 될 수 있다.
- copyreg를 통해 역직렬화시 사용할 함수에 대해 안정적인 식별자를 지정할 수 있다.

In [46]:
import pickle
import copyreg


class Student:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        self.dummy = 'dummy'  # dummy 속성을 새로 추가!


def pickle_student(student):
    kwargs = student.__dict__
    return unpickle_student, (kwargs, )


def unpickle_student(kwargs):
    return Student(**kwargs)

copyreg.pickle(Student, pickle_student)
a = Student('임철희', 27)
with open('student.p', 'wb') as f:
    pickle.dump(a, f)

copyreg.pickle(Student, pickle_student)
with open('student.p', 'rb') as f:
    student = pickle.load(f)  # unpickle_student() 함수를 호출한다.

TypeError: __init__() got an unexpected keyword argument 'dummy'